In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import morphology
from skimage.color import rgb2gray
#Might be more usable when its possible to successfully remove hair, so we can proper detect colour differention
#when it shifts from edge to skin.
def sharpness_score(mask):
    mask = cv2.imread(mask,cv2.IMREAD_GRAYSCALE)
    #mask = np.zeros((256, 256), dtype=np.uint8)
    if mask is None:
        raise FileNotFoundError("Could not load lesion_mask.png. Check the file path.")
    cv2.circle(mask, (128, 128), 60, 255, -1)
    #I try with sobel
    sobel_x = cv2.Sobel(mask, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(mask, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = cv2.magnitude(sobel_x, sobel_y)
    edge_pixels = gradient_magnitude[mask > 0]
    sharpness_score = np.mean(edge_pixels)


    return sharpness_score

file_1 = "/Users/simonbruun-simonsen/Desktop/FeatureExtraction/2025-FYP-groupKangaroo/data/PAT_161_250_197_mask.png"
file_2 = "/Users/simonbruun-simonsen/Desktop/FeatureExtraction/2025-FYP-groupKangaroo/data/PAT_168_260_654_mask.png"
file_3 = "/Users/simonbruun-simonsen/Desktop/FeatureExtraction/2025-FYP-groupKangaroo/data/PAT_43_61_210_mask.png"
sharpness_score(file_1)




24.32276093114595

In [2]:
from skimage import measure
def compactness(mask_file):
    #Reads file
    mask = cv2.imread(mask_file)
    #Makes it grayscaled, just in case.
    mask_gray = rgb2gray(mask)
    #Counts all pixels brigther than 0.5
    mask_bin = mask_gray > 0.5
    
    #We sum all the true values.
    A = np.sum(mask_bin)

    #We use a morphology disk
    struct = morphology.disk(2)
    
    #mask_eroded  = morphology.binary_erosion(mask_bin, struct)

    #perimeter = mask_bin & ~mask_eroded# Whats left after subtracting our mask from the eroded mask
    #L = np.sum(perimeter)
    #Returns total perimeter of all objects in BINARY image.
    L = measure.perimeter(mask_bin)
    #migth get easily influenced on the perimeter.
    compactness = (4*np.pi*A) / (L**2)

    return compactness


file_4 = "/Users/simonbruun-simonsen/Desktop/FeatureExtraction/2025-FYP-groupKangaroo/data/PAT_39_55_233_mask.png"
file_5 = "/Users/simonbruun-simonsen/Desktop/FeatureExtraction/2025-FYP-groupKangaroo/data/PAT_54_83_405_mask.png"
compactness(file_4)




0.892648663992436

In [22]:
from scipy.spatial import ConvexHull

#Irregulairity index, starts from 1, mostly goes up till 1.5 if no noise.
def convexity(mask):
    #Reads file
    mask = cv2.imread(mask)
    #Makes it grayscaled, just in case.
    mask = rgb2gray(mask)
    coords = np.column_stack(np.nonzero(mask))
    hull = ConvexHull(coords)
    hull_coords = coords[hull.vertices]
    perimeter = measure.perimeter(mask)
    
    hull_perimeter = np.sum(np.sqrt(np.sum(np.diff(np.vstack([hull_coords, hull_coords[0]]), axis = 0)**2, axis = 1)))
    convex_score = perimeter/hull_perimeter
    #Score at around 1.0 means PERFECT circle.
    #Scores at 1.5 ish its jagged. Above that we could say its influenced by noise.
    return convex_score

convexity(file_2)


1.0743759851944519

"While alternative border-based irregularity measures were considered (e.g., contour simplification with approxPolyDP), these were found to be overly sensitive to noise and often failed to capture the overall shape complexity. The convex hull area-based index, despite its limitations in ignoring indentations, offered a more stable and interpretable metric across our dataset."

In [20]:
from cv2 import approxPolyDP


def Poly(mask):
    #Reads file
    mask = cv2.imread(mask)
    #Makes it grayscaled, just in case.
    mask = rgb2gray(mask)
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Get the largest contour (assumes the lesion is the largest)
    main_contour = max(contours, key=cv2.contourArea)
    
    # Calculate the area of the original contour
    area_original = cv2.contourArea(main_contour)
    
    # Calculate the convex hull of the original contour
    convex_hull = cv2.convexHull(main_contour)
    
    # Calculate the area of the convex hull
    area_convex = cv2.contourArea(convex_hull)
    
    # Calculate the irregularity index based on area difference
    irregularity_index = abs(area_convex - area_original) / area_convex
    
    #irregularity_index = abs(perimeter_original-perimeter_approx) / area_original
    # Draw the original and approximated contours on the mask (for visualization)
    img_copy = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)  # Make the mask 3-channel for drawing
    cv2.drawContours(img_copy, [main_contour], -1, (0, 255, 0), 2)  # Green for original
    cv2.drawContours(img_copy, [convex_hull], -1, (0, 0, 255), 2)  # Red for approximated

    cv2.imshow("Contour and Approximation", img_copy)
    cv2.waitKey(0)  # Wait until a key is pressed
    cv2.destroyAllWindows()  # Close the window when done
    
    return img_copy, irregularity_index, area_original, area_convex

Poly(file_3)

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 0.2438593060842619,
 305500.5,
 404026.0)

In [27]:
#Area based, instead of parameter based.
def solidity(mask):
        #Reads file
    mask = cv2.imread(mask)
    #Makes it grayscaled, just in case.
    mask = rgb2gray(mask)
    mask = mask.astype(np.uint8)

    coords = np.column_stack(np.nonzero(mask))
    hull = ConvexHull(coords)
    lesion_area = np.count_nonzero(mask) # only count 1s
    hull_area = hull.volume #returns are of hull in scipy. Bit misleading.
    solidity_score = lesion_area/hull_area
    return solidity_score

solidity(file_4)


0.9995277580419404